In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip /content/gdrive/MyDrive/Data.zip

Streaming output truncated to the last 5000 lines.
  inflating: Data/train/open/s0036_04174_1_0_1_0_1_01.png  
  inflating: Data/train/open/s0036_04175_1_0_1_0_1_01.png  
  inflating: Data/train/open/s0036_04176_1_0_1_0_1_01.png  
  inflating: Data/train/open/s0036_04178_1_0_1_0_0_01.png  
  inflating: Data/train/open/s0036_04179_1_0_1_0_0_01.png  
  inflating: Data/train/open/s0036_04180_1_0_1_0_0_01.png  
  inflating: Data/train/open/s0036_04182_1_0_1_0_0_01.png  
  inflating: Data/train/open/s0036_04183_1_0_1_0_1_01.png  
  inflating: Data/train/open/s0036_04184_1_0_1_0_0_01.png  
  inflating: Data/train/open/s0036_04185_1_0_1_0_1_01.png  
  inflating: Data/train/open/s0036_04186_1_0_1_0_0_01.png  
  inflating: Data/train/open/s0036_04187_1_0_1_0_1_01.png  
  inflating: Data/train/open/s0036_04188_1_0_1_0_0_01.png  
  inflating: Data/train/open/s0036_04189_1_0_1_0_1_01.png  
  inflating: Data/train/open/s0036_04190_1_0_1_0_0_01.png  
  inflating: Data/train/open/s0036_04193_1_0_1_1_

In [4]:
BATCH_SIZE = 4
EPOCHS = 2

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 0.2,shear_range = 0.2,
    zoom_range = 0.2,width_shift_range = 0.2,
    height_shift_range = 0.2, validation_split = 0.2)

train_data= train_datagen.flow_from_directory(os.path.join('Data', 'train'),
                                target_size = (80,80), batch_size = BATCH_SIZE, 
                                class_mode = 'categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(os.path.join('Data', 'train'),
                                target_size = (80,80), batch_size = BATCH_SIZE, 
                                class_mode = 'categorical', subset='validation')

Found 54336 images belonging to 2 classes.
Found 13583 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_data = test_datagen.flow_from_directory(os.path.join('Data', 'test'),
                                target_size=(80,80), batch_size = BATCH_SIZE, class_mode='categorical')

Found 16979 images belonging to 2 classes.


In [7]:
bmodel = InceptionV3(include_top = False, weights = 'imagenet', 
                     input_tensor = Input(shape = (80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation = 'relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation = 'softmax')(hmodel)

model = Model(inputs = bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 5s 0us/step


In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [10]:
checkpoint = ModelCheckpoint(os.path.join("model.h5"),
                            monitor = 'val_loss', save_best_only = True, verbose = 3)

earlystop = EarlyStopping(monitor = 'val_loss', patience = 7, 
                          verbose= 3, restore_best_weights = True)


learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks = [checkpoint, earlystop, learning_rate]

In [11]:
model.compile(optimizer = 'Adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])


model.fit_generator(train_data,steps_per_epoch = train_data.samples// BATCH_SIZE,
                   validation_data = validation_data,
                   validation_steps = validation_data.samples// BATCH_SIZE,
                   callbacks = callbacks,
                    epochs = EPOCHS)

<ipython-input-11-698b6b67be90>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch = train_data.samples// BATCH_SIZE,


Epoch 1/2
13581/13584 [============================>.] - ETA: 0s - loss: 0.2095 - accuracy: 0.9180
Epoch 1: val_loss improved from inf to 0.26440, saving model to model.h5
13584/13584 [==============================] - 351s 25ms/step - loss: 0.2095 - accuracy: 0.9180 - val_loss: 0.2644 - val_accuracy: 0.8974 - lr: 0.0010
Epoch 2/2
13583/13584 [============================>.] - ETA: 0s - loss: 0.1860 - accuracy: 0.9276
Epoch 2: val_loss improved from 0.26440 to 0.24469, saving model to model.h5
13584/13584 [==============================] - 358s 26ms/step - loss: 0.1860 - accuracy: 0.9276 - val_loss: 0.2447 - val_accuracy: 0.8968 - lr: 0.0010


## Model Evaluation

In [14]:
# Training Data 
acc_tr, loss_tr = model.evaluate_generator(train_data)
print("Accuracy of Training: ", acc_tr)
print("Loss at Training: ", loss_tr)

<ipython-input-14-562a6561e778>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


Accuracy of Training:  0.15530188381671906
Loss at Training:  0.9406654834747314


In [15]:
# Validation Data 
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print("Accuracy of Training: ", acc_tr)
print("Loss at Training: ", loss_tr)

<ipython-input-15-5946541904dd>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


Accuracy of Training:  0.15530188381671906
Loss at Training:  0.9406654834747314


In [16]:
# Test Data 
acc_test, loss_test = model.evaluate_generator(test_data)
print("Accuracy of Training: ", acc_tr)
print("Loss at Training: ", loss_tr)

<ipython-input-16-0822c36ff8c5>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


Accuracy of Training:  0.15530188381671906
Loss at Training:  0.9406654834747314
